## *We will be using Exchange Rate API*

In [45]:
from langchain_core.tools import tool, InjectedToolArg
import requests
from typing import Annotated

In [47]:
# CREATING TOOL

@tool
def conversion_factor(base_currency: str, target_currency: str):
    """
    This function fetches the currency factors between a given base currecny and target currency
    """

    url = f'https://v6.exchangerate-api.com/v6/78bcba80780d064f72d9dd66/pair/{base_currency}/{target_currency}'
    response = requests.get(url)

    return response.json()


# @tool
# def convert(base_currency_value: int, conversion_rate: float) -> float:
#     """
#     Provide Conversion from one currency rate to another
 
#     """

#     return base_currency_value * conversion_rate


@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float , InjectedToolArg]) -> float:
    """
    Provide Conversion from one currency rate to another
 
    """

    return base_currency_value * conversion_rate


In [49]:
result = conversion_factor.invoke({
    "base_currency": "USD",
    "target_currency": "PKR"   
})
print(result)

{'result': 'success', 'documentation': 'https://www.exchangerate-api.com/docs', 'terms_of_use': 'https://www.exchangerate-api.com/terms', 'time_last_update_unix': 1755820801, 'time_last_update_utc': 'Fri, 22 Aug 2025 00:00:01 +0000', 'time_next_update_unix': 1755907201, 'time_next_update_utc': 'Sat, 23 Aug 2025 00:00:01 +0000', 'base_code': 'USD', 'target_code': 'PKR', 'conversion_rate': 283.535}


In [50]:
convert.invoke({'base_currency_value':10, 'conversion_rate': 86.11})

861.1

In [51]:
# TOOL BINDING

from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model = 'gemini-2.0-flash'
)
llm_with_tools = llm.bind_tools([conversion_factor, convert ])

In [66]:
from langchain_core.messages import HumanMessage


messages = [HumanMessage('What is the conversion factor between USD and PKR, and based on that can you convert 100 USD to PKR')]

In [74]:
messages

[HumanMessage(content='What is the conversion factor between USD and PKR, and based on that can you convert 100 USD to PKR', additional_kwargs={}, response_metadata={})]

In [67]:
ai_message = llm_with_tools.invoke(messages)

In [68]:
ai_message

AIMessage(content='', additional_kwargs={'function_call': {'name': 'conversion_factor', 'arguments': '{"base_currency": "USD", "target_currency": "PKR"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--f9b5e3c3-e4af-4da0-9006-6e0d271595c9-0', tool_calls=[{'name': 'conversion_factor', 'args': {'base_currency': 'USD', 'target_currency': 'PKR'}, 'id': '00606e96-5f15-4981-bcd5-4b674762bb8c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 82, 'output_tokens': 12, 'total_tokens': 94, 'input_token_details': {'cache_read': 0}})

In [78]:
messages.append(ai_message)

In [80]:
messages

[HumanMessage(content='What is the conversion factor between USD and PKR, and based on that can you convert 100 USD to PKR', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'conversion_factor', 'arguments': '{"base_currency": "USD", "target_currency": "PKR"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--f9b5e3c3-e4af-4da0-9006-6e0d271595c9-0', tool_calls=[{'name': 'conversion_factor', 'args': {'base_currency': 'USD', 'target_currency': 'PKR'}, 'id': '00606e96-5f15-4981-bcd5-4b674762bb8c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 82, 'output_tokens': 12, 'total_tokens': 94, 'input_token_details': {'cache_read': 0}}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'conversion_factor', 'arguments': '{"base_currency": "USD", "target_currency": "PKR"}'}}, response_

In [85]:
ai_message.tool_calls

[{'name': 'conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'PKR'},
  'id': '00606e96-5f15-4981-bcd5-4b674762bb8c',
  'type': 'tool_call'}]

In [84]:
import json

for tool_call in ai_message.tool_calls:
    # print(tool_call)

    # Execute the first tool and get the conversion rate
    if tool_call['name'] == 'conversion_factor':
        tool_message1 = conversion_factor.invoke(tool_call)
        # Fetch this message rate
        print(json.load(tool_call.content)['conversion_rate'])
        #append this tool message to message list
        messages.append(tool_message1) 

    # Execute the 2nd tool using the conversion rate from tool1
    if tool_call['name'] == 'convert':
        # Fetch the current arg
        tool_call['args']['conversion_rate']
        tool_message2 = convert.invoke(tool_call)
        messages.append(tool_message2 )


AttributeError: 'dict' object has no attribute 'content'

In [83]:
messages

[HumanMessage(content='What is the conversion factor between USD and PKR, and based on that can you convert 100 USD to PKR', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'conversion_factor', 'arguments': '{"base_currency": "USD", "target_currency": "PKR"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--f9b5e3c3-e4af-4da0-9006-6e0d271595c9-0', tool_calls=[{'name': 'conversion_factor', 'args': {'base_currency': 'USD', 'target_currency': 'PKR'}, 'id': '00606e96-5f15-4981-bcd5-4b674762bb8c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 82, 'output_tokens': 12, 'total_tokens': 94, 'input_token_details': {'cache_read': 0}}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'conversion_factor', 'arguments': '{"base_currency": "USD", "target_currency": "PKR"}'}}, response_

### `After new update: LangChain won’t automatically run the second tool after the first.`